In this notebook, I experiment with ReLoRa: a framework that enables pre-training with low-rank adapters that I described here:.

*Last update: July 19th 2023. If something doesn't work anymore, you will have to retrieve versions available at that date. Please also comment about the issue on the related post so that I can fix the notebook: *

In [ ]:
!git clone https://github.com/Guitaricet/peft_pretraining.git

Cloning into 'peft_pretraining'...
remote: Enumerating objects: 635, done.
remote: Counting objects: 100% (61/61), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 635 (delta 43), reused 40 (delta 29), pack-reused 574
Receiving objects: 100% (635/635), 1.20 MiB | 23.15 MiB/s, done.
Resolving deltas: 100% (398/398), done.


In [ ]:
%cd peft_pretraining/

/content/peft_pretraining


Install all the requirements:

In [ ]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 89.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.8/206.8 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Since training takes a lot of time, I recommend first trying the framework with hyper-parameters that will stop the training early and shorten the validation (which can take many hours on consumer hardware).

The framework doesn't have the option to shorten the validation, so we will have to do it manually. Open the file torchrun_main.py, and replace the line:

*if evaluated_on_tokens > target_eval_tokens:*


Note: At the time of writing this article, this line is line 129.
by:


*if evaluated_on_tokens > target_eval_tokens or total_batches > 10:*

Step one: We initialize pre-training without peft and without LoRa

In [ ]:
!torchrun --nproc-per-node 1 torchrun_main.py \
    --model_config configs/llama_250m.json \
    --batch_size 4 \
    --total_batch_size 8 \
    --lr 5e-4 \
    --max_length 512 \
    --tags warm_start_250M \
    --save_every 10 \
    --num_training_steps 2 \
    --workers 1 \
    --eval_every 1

2023-07-19 09:39:06.210043: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Starting script
local rank: 0, device: 0
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: 3
wandb: You chose "Don't visualize my results"
wandb: Tracking run with wandb version 0.15.5
wandb: W&B syncing is set to `offline` in this directory.  
wandb: Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.
2023-07-19 09:39:14.105 | INFO     | __main__:main:191 - Using torch.distributed with rank 0 (only rank 0 will log)
2023-07-19 09:39:14.106 | INFO     | __main__:main:192 - ****************************************
2023-07-19 09:39:14.106 | INFO     | __main__:main:193 - Starting training with the arguments
2023-07-19 09:39:14.106 | INFO     | __main__:main:195 - model_config                   configs/llama_250m.json
2023-07-19 09:39:14.106 | INFO     | __main__:main

Step two: Perform ReLoRa

In [ ]:
!torchrun --nproc-per-node 1 torchrun_main.py \
    --model_config configs/llama_250m.json \
    --batch_size 4 \
    --total_batch_size 8 \
    --lr 1e-3 \
    --max_length 512 \
    --use_peft \
    --relora 5 \
    --cycle_length 5 \
    --restart_warmup_steps 10 \
    --scheduler cosine_restarts \
    --warmup_steps 2 \
    --reset_optimizer_on_relora True \
    --num_training_steps 50 \
    --save_every 10 \
    --eval_every 10 \
    --continue_from checkpoints/llama_250m-2023-07-19-09-39-08/model_3 \ #change this line to the name of your checkpoints
    --tags relora_250M

2023-07-19 09:58:37.992710: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Starting script
local rank: 0, device: 0
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B account
wandb: (3) Don't visualize my results
wandb: Enter your choice: 3
wandb: You chose "Don't visualize my results"
wandb: Tracking run with wandb version 0.15.5
wandb: W&B syncing is set to `offline` in this directory.  
wandb: Run `wandb online` or set WANDB_MODE=online to enable cloud syncing.
2023-07-19 09:58:43.765 | INFO     | __main__:main:190 - Using torch.distributed with rank 0 (only rank 0 will log)
2023-07-19 09:58:43.766 | INFO     | __main__:main:191 - ****************************************
2023-07-19 09:58:43.766 | INFO     | __main__:main:192 - Starting training with the arguments
2023-07-19 09:58:43.766 | INFO     | __main__:main:194 - model_config                   configs/llama_250m.json
2023-07-19 09:58:43.766 | INFO     | __main__:main

Now that we are sure that everything works, we can run ReLoRa pre-training with reasonable hyperparameters. *Note: I recommend removing "total_batches > 10" that we have added to speed up validation.*

In [ ]:
torchrun --nproc-per-node 1 torchrun_main.py \
    --model_config configs/llama_250m.json \
    --batch_size 4 \
    --total_batch_size 8\
    --lr 5e-4 \
    --max_length 512 \
    --tags warm_start_250M \
    --save_every 1000 \
    --num_training_steps 10000

torchrun --nproc-per-node 1 torchrun_main.py \
    --model_config configs/llama_250m.json \
    --batch_size 4 \
    --total_batch_size 8 \
    --lr 1e-3 \
    --max_length 512 \
    --use_peft \
    --relora 5000 \
    --cycle_length 5000 \
    --restart_warmup_steps 100 \
    --scheduler cosine_restarts \
    --warmup_steps 500 \
    --reset_optimizer_on_relora True \
    --num_training_steps 10000 \
    --save_every 5000 \
    --eval_every 5000 \
    --continue_from <your checkpoint from step one>\ #change this line to the name of your checkpoints
    --tags relora_250M